In [1]:
## extract feature from original image and sobel filtered image
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from scipy.sparse import dok_matrix,coo_matrix,csr_matrix,vstack,save_npz

import pandas as pd
import sys
from pathlib2 import Path
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
import pickle
from joblib import dump, load

feature:
0 original values (HU)
1 neighbouring mean
2 neighbouring sobel
3 square
4 3d neighbouring mean

In [2]:
N = 0

# N = 1258153686 got easily
# N_val = 381672737
# N_train = 876480949
data_all = torch.empty([1258153686,6])
data_train = torch.empty([876480949,6])
data_val = torch.empty([381672737,7])
# data_all = coo_matrix([0,0,0,0,1])
# data_all = dok_matrix((N,5),dtype=np.float32)
maxpool = nn.MaxPool2d(kernel_size=(13,13),stride=(1,1),padding=(6,6))
avgpool = nn.AvgPool2d(kernel_size=(13,13),stride=(1,1),padding=(6,6))
maxpool3d = nn.MaxPool3d(kernel_size=(7,13,13),stride=(1,1,1),padding=(3,6,6))
# target = np.zeros([N,1])


for i in range(63,210):
# for i in range(63):
    img_path = f'./data/case_00{i:03}/imaging/'
    seg_path = f'./data/case_00{i:03}/segmentation/'
    slice_len = int(len([d for d in Path(img_path).iterdir()])/2)
    print(i)
    
    img_3d = torch.empty([slice_len,248,248]).cuda()
    N_j = N
    N_slice = slice_len
    for j in range(slice_len):
        img = np.load(str(img_path)+str(f'{j:03}.npy'))
        if img.sum()==0:
            N_slice = N_slice-1
            continue
        sob = np.load(str(img_path)+str(f'{j:03}_sobel.npy'))
        seg = np.load(str(seg_path)+str(f'{j:03}.npy'))
        
        feature0 = torch.cuda.HalfTensor(img)
        img_3d[j,:feature0.shape[0],:feature0.shape[1]] = feature0
        
        feature1 = avgpool(torch.cuda.HalfTensor(img).unsqueeze(0))
        feature2 = avgpool(torch.cuda.HalfTensor(sob).unsqueeze(0))
        feature3 = (torch.cuda.HalfTensor(img).unsqueeze(0)/(maxpool(torch.cuda.HalfTensor(img).abs().unsqueeze(0))+1e-8))**2
        
#         if torch.sum(torch.isnan(feature3)).item()>0:
#             print(index_nan)
        target = torch.cuda.HalfTensor(np.round((seg/1.0)))
        features_temp = torch.concat((feature0.view(-1,1),feature1.view(-1,1),feature2.view(-1,1),feature3.view(-1,1)),1)
        id_val = torch.ones([features_temp.shape[0],1]).cuda()*i

        data_train[N:N+len(features_temp),:4] = features_temp.cpu()
        data_train[N:N+len(features_temp),5] = target.view(-1,1)[:,0]
        
#         data_val[N:N+len(features_temp),:4] = features_temp.cpu()
#         data_val[N:N+len(features_temp),5] = target.view(-1,1)[:,0]
#         data_val[N:N+len(features_temp),6] = id_val.view(-1,1)[:,0]

        N = N+len(features_temp)
    
    img_3d = img_3d[:N_slice,:feature0.shape[0],:feature0.shape[1]]
    
    feature4 = maxpool3d(img_3d.unsqueeze(0))
    data_train[N_j:N,4] = feature4.view(feature4.shape[0],-1,1).view(-1,1)[:,0]
    
#     data_val[N_j:N,4] = feature4.view(feature4.shape[0],-1,1).view(-1,1)[:,0]
    
    




63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209


In [3]:
data_train.shape

torch.Size([876480949, 6])

In [ ]:
data_train = torch.unique(data_train,dim=0)
data_train.shape

In [35]:
# data_cpu = data_train.data.cpu().numpy()
data_cpu = data_val.data.cpu().numpy()

In [36]:
# np.save('data_train.npy',data_cpu)
np.save('data_val.npy',data_cpu)

In [ ]:
d = np.load("./data/case_00190/imaging/050.npy")
s= np.load("./data/case_00190/imaging/050_sobel.npy")
l = np.load("./data/case_00190/segmentation/050.npy")

In [ ]:
np.unique(l)

In [ ]:
plt.imshow(d)

In [ ]:
plt.imshow(s)

In [ ]:
print(d.shape)
plt.imshow(l)

In [ ]:
print(d.shape)
plt.imshow(d+l*1000)